Построить обобщенную линейную модель (GLM) для прогнозирования наступления страховых случаев на рассмотренных в ноутбуке данных. Подобрать необходимое распределение и тип связи, при необходимости ознакомиться с документацией H2O. Придумать и использовать дополнительные факторы при построении модели (например, пересечения признаков или функции от них и т.д.). Оценить результаты построенной модели при помощи различных метрик (можно использовать и другие метрики помимо представленных в ноутбуке), проанализировать вероятные проблемы. Предложить способы их решения и/или попробовать их решить, улучшив результат.

In [1]:
import numpy as np
import pandas as pd

import h2o

In [2]:
# Загрузим набор данных

df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [3]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 16  Cl

In [5]:
df.head(20)

,Exposure,LicAge,RecordBeg,RecordEnd,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea
0,0.083,332,2004-01-01,2004-02-01,Male,Other,CSP50,Professional,46,0,50,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,9.0
1,0.916,333,2004-02-01,NaN,Male,Other,CSP50,Professional,46,0,50,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,9.0
2,0.550,173,2004-05-15,2004-12-03,Male,Other,CSP50,Private+trip to office,32,0,68,0.0,0,0.0,2.0,0.0,0.0,0.0,0.0,7.0
3,0.089,364,2004-11-29,NaN,Female,Other,CSP55,Private+trip to office,52,0,50,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
4,0.233,426,2004-02-07,2004-05-01,Male,Other,CSP60,Private,57,0,50,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
5,0.666,429,2004-05-01,NaN,Male,Other,CSP60,Private,57,0,50,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
6,0.080,461,2004-04-02,2004-05-01,Male,Other,CSP48,Professional,58,0,50,0.0,0,0.0,1.0,0.0,0.0,2.0,1.0,5.0
7,0.666,462,2004-05-01,NaN,Male,Other,CSP48,Professional,58,0,50,0.0,0,0.0,1.0,0.0,0.0,2.0,1.0,5.0
8,0.173,405,2004-10-29,NaN,Female,Other,CSP50,Private+trip to office,53,0,50,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
9,0.474,386,2004-01-01,2004-06-22,Male,Other,CSP55,Private+trip to office,57,0,50,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,6.0


In [6]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [7]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [8]:
df.loc[df.ClaimsCount > 0, 'AvgClaim'] = df['ClaimAmount']/df['ClaimsCount']

In [9]:
class InsDataFrame:


    ''' Load data method '''

    def load_pd(self, pd_dataframe):
        self._df = pd_dataframe


    ''' Columns match method '''

    def columns_match(self, match_from_to):
        self._df.rename(columns=match_from_to, inplace=True)


    ''' Person data methods '''

    # Gender
    _gender_dict = {'Male':0, 'Female':1}

    def transform_gender(self):
        self._df['Gender'] = self._df['Gender'].map(self._gender_dict)

        

    # Age
    @staticmethod
    def _age(age, age_max):
        if pd.isnull(age):
            age = None
        elif age < 18:
            age = None
        elif age > age_max:
            age = age_max
        return age
      
    def transform_age(self, age_max=70):
        self._df['driver_minage'] = self._df['driver_minage'].apply(self._age, args=(age_max,))

    # Age M/F
    @staticmethod
    def _age_gender(age_gender):
        _age = age_gender[0]
        _gender = age_gender[1]
        if _gender == 0: #Male
            _driver_minage_m = _age
            _driver_minage_f = 18
        elif _gender == 1: #Female
            _driver_minage_m = 18
            _driver_minage_f = _age
        else:
            _driver_minage_m = 18
            _driver_minage_f = 18
        return [_driver_minage_m, _driver_minage_f]
    
    def transform_age_gender(self):
        self._df['driver_minage_m'],self._df['driver_minage_f'] = zip(*self._df[['driver_minage','Gender']].apply(self._age_gender, axis=1).to_frame()[0])

    # Experience
    @staticmethod
    def _exp(exp, exp_max):
        if pd.isnull(exp):
            exp = None
        elif exp < 0:
            exp = None
        elif exp > exp_max:
            exp = exp_max
        return exp

    def transform_exp(self, exp_max=52):
        self._df['driver_minexp'] = self._df['driver_minexp'].apply(self._exp, args=(exp_max,))


    ''' Other data methods '''

    def polynomizer(self, column, n=2):
        if column in list(self._df.columns):
            for i in range(2,n+1):
                self._df[column+'_'+str(i)] = self._df[column]**i

    def get_dummies(self, columns):
        self._df = pd.get_dummies(self._df, columns=columns)
        
    
    # Вспомогательный столбец для суммирования числа полисов с убытками
    #df['ClaimCount'] = df['ClaimAmount'] > 0


    ''' General methods '''

    def info(self):
        return self._df.info()

    def head(self, columns, n=5):
        return self._df.head(n)

    def len(self):
        return len(self._df)

    def get_pd(self, columns):
        return self._df[columns]
    
class InsDataFrame_Fr(InsDataFrame):

    # Experience (weeks to years)
    @staticmethod
    def _exp(exp, exp_max):
        if pd.isnull(exp):
            exp = None
        elif exp < 0:
            exp = None
        else:
            exp * 7 // 365
        if exp > exp_max:
            exp = exp_max
        return exp

    # Marital status
    _MariStat_dict = {'Other':0, 'Alone':1}

    def transform_MariStat(self):
        self._df['MariStat'] = self._df['MariStat'].map(self._MariStat_dict)
    
    # Social category
    def transform_SocioCateg(self):
        self._df['SocioCateg'] = self._df['SocioCateg'].str.slice(0,4)

In [10]:
data = InsDataFrame_Fr()

In [11]:
data.load_pd(df)

In [12]:
# Переименовываем
data.columns_match({'DrivAge':'driver_minage','LicAge':'driver_minexp'})

In [13]:
# Преобразовываем
data.transform_age()
data.transform_exp()
data.transform_gender()
data.transform_MariStat()
data.transform_SocioCateg()

In [14]:
# Пересечение пола и возраста, их квадраты
data.transform_age_gender()
data.polynomizer('driver_minage_m')
data.polynomizer('driver_minage_f')

In [15]:
data.get_dummies(['VehUsage','SocioCateg'])

In [16]:
data.get_dummies(['RiskArea'])

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 42 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Exposure                         115155 non-null  float64
 1   driver_minexp                    115155 non-null  int64  
 2   RecordBeg                        115155 non-null  object 
 3   RecordEnd                        59455 non-null   object 
 4   Gender                           115155 non-null  int64  
 5   MariStat                         115155 non-null  int64  
 6   driver_minage                    115155 non-null  int64  
 7   HasKmLimit                       115155 non-null  int64  
 8   BonusMalus                       115155 non-null  int64  
 9   ClaimAmount                      115155 non-null  float64
 10  ClaimInd                         115155 non-null  int64  
 11  OutUseNb                         115155 non-null  float64
 12  Cl

In [18]:
col_features = [
                'driver_minexp',
                'Gender',
                'MariStat',
                'HasKmLimit',
                'BonusMalus',
                'OutUseNb',
                'driver_minage_m',
                'driver_minage_f',
                'driver_minage_m_2',
                'driver_minage_f_2',
                'VehUsage_Private',
                'VehUsage_Private+trip to office',
                'VehUsage_Professional',
                'VehUsage_Professional run',
                'SocioCateg_CSP1',
                'SocioCateg_CSP2',
                'SocioCateg_CSP3',
                'SocioCateg_CSP4',
                'SocioCateg_CSP5',
                'SocioCateg_CSP6',
                'SocioCateg_CSP7',
                'RiskArea_1.0',
                'RiskArea_2.0',
                'RiskArea_3.0',
                'RiskArea_4.0',
                'RiskArea_5.0',
                'RiskArea_6.0',
                'RiskArea_7.0',
                'RiskArea_8.0',
                'RiskArea_9.0',
                'RiskArea_10.0',
                'RiskArea_11.0',
                'RiskArea_12.0',
                'RiskArea_13.0',
    
]

In [19]:
col_target = ['ClaimInd']

In [20]:
df = data.get_pd(col_features+col_target)

In [21]:
h2o.init()

C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\connection.py:368: UserWarning: Proxy is defined in the environment: HTTP_PROXY. This may interfere with your H2O Connection.
  warn("Proxy is defined in the environment: %s. "


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ADA90~1.KRA\AppData\Local\Temp\tmpvj5crs5t
  JVM stdout: C:\Users\ADA90~1.KRA\AppData\Local\Temp\tmpvj5crs5t\h2o_a_kraev_started_from_python.out
  JVM stderr: C:\Users\ADA90~1.KRA\AppData\Local\Temp\tmpvj5crs5t\h2o_a_kraev_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Yekaterinburg
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,"7 days, 3 hours and 9 minutes"
H2O_cluster_name:,H2O_from_python_a_kraev_k5g63j
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.754 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [22]:
from sklearn.model_selection import train_test_split
# Разбиение датасета на train/val/test
x_train, x_test, y_train, y_test = train_test_split(df[col_features], df.ClaimInd, test_size=0.3, random_state=1)
x_valid, x_test, y_valid, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=1)

# Преобразование в H2O-Frame
h2o_train = h2o.H2OFrame(pd.concat([x_train, y_train], axis=1))
h2o_valid = h2o.H2OFrame(pd.concat([x_valid, y_valid], axis=1))
h2o_test = h2o.H2OFrame(pd.concat([x_test, y_test], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [23]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [24]:
h2o_train['ClaimInd'] = h2o_train['ClaimInd'].asfactor()
h2o_valid['ClaimInd'] = h2o_valid['ClaimInd'].asfactor()
h2o_test['ClaimInd'] = h2o_test['ClaimInd'].asfactor()

In [25]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm = H2OGeneralizedLinearEstimator(family='binomial', nfolds=5)
glm.train(y="ClaimInd", x = h2o_train.names[1:-1], training_frame = h2o_train, validation_frame = h2o_valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [26]:
glm.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 2.368E-5 )",33,32,3,py_1_sid_85a6


In [27]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.52576256,0.099619746,0.5933615,0.56775963,0.55814815,0.3493327,0.56021076
1,auc,0.5648464,0.0056655584,0.5626205,0.5659478,0.5665174,0.55685025,0.57229596
2,aucpr,0.1146994,0.004572776,0.11005195,0.110369645,0.11864625,0.114469975,0.11995918
3,err,0.47423747,0.099619746,0.4066385,0.43224037,0.44185185,0.6506673,0.43978927
4,err_count,7654.8,1677.6321,6542.0,6934.0,7158.0,10628.0,7012.0
5,f0point5,0.1337142,0.0041985665,0.13111062,0.13128066,0.13823232,0.12965725,0.1382901
6,f1,0.18717563,0.0051112655,0.18081643,0.18288946,0.19155183,0.1884545,0.1921659
7,f2,0.31281105,0.020169329,0.291223,0.30135924,0.3118335,0.3448276,0.314812
8,lift_top_group,1.349285,0.35468224,0.9516711,1.7707679,1.3282733,1.0588859,1.6368265
9,logloss,0.3111364,0.007356746,0.30391562,0.3032413,0.3174386,0.3189679,0.3121185


In [28]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.394159e+00,-2.281654
1,Gender,-6.496232e-02,-0.031487
2,MariStat,-6.639783e-02,-0.023919
3,HasKmLimit,-3.687965e-01,-0.115171
4,BonusMalus,6.759236e-03,0.103756
5,OutUseNb,5.978640e-02,0.041594
6,driver_minage_m,-3.881190e-03,-0.073085
7,driver_minage_f,-1.119175e-03,-0.017951
8,driver_minage_m_2,-5.098931e-06,-0.007961
9,driver_minage_f_2,-2.852032e-07,-0.000359


In [29]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm.coef_norm()
for x in range(len(glm.cross_validation_models())):
    pmodels[x] = glm.cross_validation_models()[x].coef_norm()
coef = pd.DataFrame.from_dict(pmodels).round(5)
coef['overall'] = abs(coef['overall'])
coef.sort_values('overall',ascending=False)

,overall,0,1,2,3,4
Intercept,2.28165,-2.27300,-2.27170,-2.29038,-2.29277,-2.28259
HasKmLimit,0.11517,-0.11428,-0.11603,-0.11533,-0.11227,-0.11718
BonusMalus,0.10376,0.10005,0.10492,0.10300,0.11083,0.09821
VehUsage_Professional,0.07413,0.07085,0.07692,0.07342,0.07856,0.07299
driver_minage_m,0.07308,-0.10395,-0.07124,-0.07305,-0.07796,-0.05598
VehUsage_Private,0.07028,-0.07185,-0.07691,-0.06810,-0.07249,-0.05893
OutUseNb,0.04159,0.04061,0.04104,0.05180,0.03348,0.04082
VehUsage_Professional run,0.04001,0.04315,0.04383,0.03298,0.03501,0.04511
RiskArea_4.0,0.03569,0.03949,0.02474,0.03728,0.04400,0.03602
Gender,0.03149,-0.03266,-0.02508,-0.03804,-0.02585,-0.02829


In [30]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

train_pred = glm.predict(h2o_train).as_data_frame()
valid_pred = glm.predict(h2o_valid).as_data_frame()
test_pred = glm.predict(h2o_test).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [31]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [32]:
print(accuracy_score(y_train, train_pred['predict']))
print(f1_score(y_train, train_pred['predict']))
print(confusion_matrix(y_train, train_pred['predict']))

0.4941196903533148
0.18658740923960745
[[35153 37820]
 [ 2958  4677]]


In [33]:
print(accuracy_score(y_valid, valid_pred['predict']))
print(f1_score(y_valid, valid_pred['predict']))
print(confusion_matrix(y_valid, valid_pred['predict']))

0.4959184854975974
0.18420312939192354
[[7583 8081]
 [ 626  983]]


In [34]:
print(accuracy_score(y_test, test_pred['predict']))
print(f1_score(y_test, test_pred['predict']))
print(confusion_matrix(y_test, test_pred['predict']))

0.4922426768553896
0.18416891451957956
[[7513 8136]
 [ 635  990]]


Необходимо упростить модель. 
Построить корреляционную матрицу и оставить 5-8 признаков с наибольшей корреляцией.